In [1]:
rm(list=ls())
gc()
setwd("/scratch/AG_Ohler/CheWei/kallisto-bus/output")

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,530421,28.4,1188254,63.5,654468,35.0
Vcells,1018462,7.8,8388608,64.0,1772256,13.6


In [2]:
as.numeric(system("awk '/MemFree/ {print $2}' /proc/meminfo", intern=TRUE))

[1] 3066455620

In [3]:
suppressMessages(library(Matrix))
suppressMessages(library(ggplot2))
suppressMessages(library(scales))
suppressMessages(library(rjson))
suppressMessages(library(R2HTML))
suppressMessages(library(Seurat))
suppressMessages(library(DoubletFinder))
suppressMessages(library(plotly))
suppressMessages(library(future))
suppressMessages(library(grid))
suppressMessages(library(tidyverse))

In [4]:
## Downloaded from https://github.com/ohlerlab/scKB/supp_data/
pp.genes <- as.character(read.table("../../proj_sc/cbpsc/data/Root_sc/Protoplasting_DEgene_FC2_list.txt", header=F)$V1)

In [5]:
sessionInfo()

R version 4.0.3 (2020-10-10)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS/LAPACK: /fast/home/c/chsu/anaconda3/envs/seu3/lib/libopenblasp-r0.3.12.so

locale:
 [1] LC_CTYPE=en_US.utf-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.utf-8        LC_COLLATE=en_US.utf-8    
 [5] LC_MONETARY=en_US.utf-8    LC_MESSAGES=en_US.utf-8   
 [7] LC_PAPER=en_US.utf-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.utf-8 LC_IDENTIFICATION=C       

attached base packages:
[1] grid      stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] forcats_0.5.0       stringr_1.4.0       dplyr_1.0.2        
 [4] purrr_0.3.4         readr_1.4.0         tidyr_1.1.2        
 [7] tibble_3.0.4        tidyverse_1.3.0     future_1.20.1      
[10] plotly_4.9.2.1      DoubletFinder_2.0.3 Seurat_3.1.5       
[13] R2HTML_2.3.2        

In [6]:
use.sample <- c("sc_12", "dc1", "dc2", "sc_43", "sc_44", "sc_45", "sc_46", "sc_47", "sc_48", "sc_49", "sc_50")
use.sample

[1] "sc_12" "dc1"   "dc2"   "sc_43" "sc_44" "sc_45" "sc_46" "sc_47" "sc_48"
[10] "sc_49" "sc_50"

In [7]:
read_seu <- function(dir,sample.name) {   
  seu <- readRDS(dir) 
  seu@misc$sample.meta.data <- list(bscs)
  seu@assays$spliced_RNA <- NULL
  seu@assays$spliced_SCT <- NULL
  seu@assays$unspliced_RNA <- NULL
  seu@assays$unspliced_SCT <- NULL
  return(seu)
}

## Seurat objects of samples are put under ./COPILOT_RDS_LENINET_DR/
list.filenames <- list.files(path = "./COPILOT_RDS_LENIENT_DR/",pattern=".rds$") %>% .[match(use.sample, gsub("_COPILOT.rds","",.))]

rc.list <- list()

for (i in 1:length(list.filenames))
{
  rc.list[[i]]<-read_seu(dir = paste0("./COPILOT_RDS_LENIENT_DR/",list.filenames[i]), sample.name = use.sample[i])
}

names(rc.list) <- list.filenames %>% gsub("_COPILOT.rds","",.)

for (i in 1:length(list.filenames))
{
  rc.list[[i]]<- suppressWarnings(suppressMessages(UpdateSeuratObject(rc.list[[i]])))
}

In [8]:
rc.features <- SelectIntegrationFeatures(object.list = rc.list, nfeatures = 25000)
length(rc.features)

[1] 20393

In [9]:
rc.features <- rc.features[-c(grep("ATMG",rc.features),grep("ATCG",rc.features),sort(match(pp.genes, rc.features)))]
length(rc.features)

[1] 17882

In [10]:
rc.list

$sc_12
An object of class Seurat 
49648 features across 10293 samples within 2 assays 
Active assay: SCT (23357 features, 23357 variable features)
 1 other assay present: RNA
 2 dimensional reductions calculated: pca, umap

$dc1
An object of class Seurat 
47748 features across 3378 samples within 2 assays 
Active assay: SCT (21818 features, 21818 variable features)
 1 other assay present: RNA
 2 dimensional reductions calculated: pca, umap

$dc2
An object of class Seurat 
47752 features across 3283 samples within 2 assays 
Active assay: SCT (22007 features, 22007 variable features)
 1 other assay present: RNA
 2 dimensional reductions calculated: pca, umap

$sc_43
An object of class Seurat 
48045 features across 8874 samples within 2 assays 
Active assay: SCT (22534 features, 22534 variable features)
 1 other assay present: RNA
 2 dimensional reductions calculated: pca, umap

$sc_44
An object of class Seurat 
47228 features across 6843 samples within 2 assays 
Active assay: SCT (22046 

In [11]:
rc.list <- PrepSCTIntegration(object.list = rc.list, anchor.features = rc.features, verbose = TRUE)

In [12]:
# Use sc_12 as reference
rc.anchors <- suppressMessages(FindIntegrationAnchors(object.list = rc.list, normalization.method = "SCT", 
    anchor.features = rc.features, verbose = TRUE, reference=1))

Warning message in CheckDuplicateCellNames(object.list = object.list):
“Some cell names are duplicated across objects provided. Renaming to enforce unique cell names.”


In [13]:
rc.integrated <- IntegrateData(anchorset = rc.anchors, normalization.method = "SCT", verbose = TRUE)
rc.integrated <- RunPCA(rc.integrated, npcs = 50, verbose = FALSE, approx = FALSE)
rc.integrated <- suppressMessages(RunUMAP(rc.integrated, reduction = "pca", dims = 1:50, umap.method = "umap-learn", metric = "correlation", n.components = 3))
rc.integrated@reductions$umap_3D <- rc.integrated@reductions$umap
rc.integrated <- suppressMessages(RunUMAP(rc.integrated, reduction = "pca", umap.method = "umap-learn", metric = "correlation", dims = 1:50))
rc.integrated@reductions$umap_2D <- rc.integrated@reductions$umap
rc.integrated <- FindNeighbors(rc.integrated, reduction = "pca",dims = 1:50)
rc.integrated <- FindClusters(rc.integrated, resolution = 5, algorithm = 3)

Integrating dataset 2 with reference dataset

Finding integration vectors

Finding integration vector weights

Integrating data

Integrating dataset 3 with reference dataset

Finding integration vectors

Finding integration vector weights

Integrating data

Integrating dataset 4 with reference dataset

Finding integration vectors

Finding integration vector weights

Integrating data

Integrating dataset 5 with reference dataset

Finding integration vectors

Finding integration vector weights

Integrating data

Integrating dataset 6 with reference dataset

Finding integration vectors

Finding integration vector weights

Integrating data

Integrating dataset 7 with reference dataset

Finding integration vectors

Finding integration vector weights

Integrating data

Integrating dataset 8 with reference dataset

Finding integration vectors

Finding integration vector weights

Integrating data

Integrating dataset 9 with reference dataset

Finding integration vectors

Finding integration ve

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 69875
Number of edges: 2111867

Running smart local moving algorithm...
Maximum modularity in 10 random starts: 0.8749
Number of communities: 116
Elapsed time: 35 seconds


In [14]:
afm <- as.matrix(rc.integrated@assays$integrated@data)
pooled <- matrix(nrow=nrow(afm), ncol = 0)
for (i in 0:(length(unique(rc.integrated@meta.data$seurat_clusters))-1)) {
m <- afm[,which(rc.integrated@meta.data$seurat_clusters==i)]
pooled <- cbind(pooled, rowSums(m)/ncol(m))
}

In [15]:
    load(file="../../cellranger/Root_bulk_arabidopsis_curated.RD")    
    time <- Reduce(merge.rownames, list(time,pooled))
    celltype <- Reduce(merge.rownames, list(celltype,pooled)) 
  
    time_label=c("Elongation", "Maturation", "Meristem")
    celltype_label=c("phloem & companion cells", "developing cortex", "hair cells", "matured cortex",
                     "matured endodermis", "non-hair cells", "columella", "phloem pole pericycle", 
                     "matured xylem pole", "protophloem & metaphloem","developing xylem", "endodermis & QC cells", "LRC & non-hair cells","QC cells")
    
    time_stat <- suppressWarnings(sapply(4:ncol(time), function(i) sapply(1:3, function(j) cor.test(time[,i],time[,j],method = "spearman")[c(3,4)])))
    time_cor <- time_stat[seq(2,nrow(time_stat),2),]
    time_pvalue <- time_stat[seq(1,nrow(time_stat)-1,2),]                                                                                                        
    time_max <- sapply(1:(ncol(time)-3), function(i) max(as.numeric(time_cor[,i])))
    time_ident <- sapply(1:(ncol(time)-3), function(i) time_label[which(as.numeric(time_cor[,i])==max(as.numeric(time_cor[,i])))])
    time_maxp <- sapply(1:(ncol(time)-3), function(i) as.numeric(time_pvalue[,i])[which(as.numeric(time_cor[,i])==max(as.numeric(time_cor[,i])))])                     
    names(time_max) <- time_ident

    celltype_stat <- suppressWarnings(sapply(15:ncol(celltype), function(i) sapply(1:14, function(j) cor.test(celltype[,i],celltype[,j],method = "spearman")[c(3,4)])))
    celltype_cor <- celltype_stat[seq(2,nrow(celltype_stat),2),]
    celltype_pvalue <- celltype_stat[seq(1,nrow(celltype_stat)-1,2),]                        
    celltype_max <- sapply(1:(ncol(celltype)-14), function(i) max(as.numeric(celltype_cor[,i])))
    celltype_ident <- sapply(1:(ncol(celltype)-14), function(i) celltype_label[which(as.numeric(celltype_cor[,i])==max(as.numeric(celltype_cor[,i])))])
    celltype_maxp <- sapply(1:(ncol(celltype)-14), function(i) as.numeric(celltype_pvalue[,i])[which(as.numeric(celltype_cor[,i])==max(as.numeric(celltype_cor[,i])))])   
    names(celltype_max) <- celltype_ident

    seucluster.celltype.ID <- celltype_ident
    seucluster.timezone.ID <- time_ident
    seucluster.celltype.cor <- celltype_max
    seucluster.timezone.cor <- time_max
    seucluster.celltype.pvalue <- celltype_maxp
    seucluster.timezone.pvalue <- time_maxp

    time_stat <- suppressWarnings(sapply(4:ncol(time), function(i) sapply(1:3, function(j) cor.test(time[,i],time[,j],method = "pearson")[c(3,4)])))
    time_cor <- time_stat[seq(2,nrow(time_stat),2),]
    time_pvalue <- time_stat[seq(1,nrow(time_stat)-1,2),]                                                                                                        
    time_max <- sapply(1:(ncol(time)-3), function(i) max(as.numeric(time_cor[,i])))
    time_ident <- sapply(1:(ncol(time)-3), function(i) time_label[which(as.numeric(time_cor[,i])==max(as.numeric(time_cor[,i])))])
    time_maxp <- sapply(1:(ncol(time)-3), function(i) as.numeric(time_pvalue[,i])[which(as.numeric(time_cor[,i])==max(as.numeric(time_cor[,i])))])                     
    names(time_max) <- time_ident

    celltype_stat <- suppressWarnings(sapply(15:ncol(celltype), function(i) sapply(1:14, function(j) cor.test(celltype[,i],celltype[,j],method = "pearson")[c(3,4)])))
    celltype_cor <- celltype_stat[seq(2,nrow(celltype_stat),2),]
    celltype_pvalue <- celltype_stat[seq(1,nrow(celltype_stat)-1,2),]                        
    celltype_max <- sapply(1:(ncol(celltype)-14), function(i) max(as.numeric(celltype_cor[,i])))
    celltype_ident <- sapply(1:(ncol(celltype)-14), function(i) celltype_label[which(as.numeric(celltype_cor[,i])==max(as.numeric(celltype_cor[,i])))])
    celltype_maxp <- sapply(1:(ncol(celltype)-14), function(i) as.numeric(celltype_pvalue[,i])[which(as.numeric(celltype_cor[,i])==max(as.numeric(celltype_cor[,i])))])   
    names(celltype_max) <- celltype_ident

    seucluster.celltype.ID.P <- celltype_ident
    seucluster.timezone.ID.P <- time_ident
    seucluster.celltype.cor.P <- celltype_max
    seucluster.timezone.cor.P <- time_max
    seucluster.celltype.pvalue.P <- celltype_maxp
    seucluster.timezone.pvalue.P <- time_maxp
                
    time <- Reduce(merge.rownames, list(Long,pooled))
    celltype <- Reduce(merge.rownames, list(Rad,pooled)) 
                             
    time_label=c("Columella", "Meri-1", "Meri-2", "Meri-3", "Meri-4", "Meri-5", "Meri-6", "Elong-7", "Elong-8", "Mat-9", "Mat-10", "Mat-11", "Mat-12")
    celltype_label=c("QC", "Hair Cell", "Cortex", "Non-Hair Cell", "Xylem Pole Pericycle", "LRC", 
                 "Columella", "Phloem Pole Pericycle", "Mat.Xylem", "Meri.Xylem", "Phloem [S32]", "Endodermis", "Phloem [SUC2]")

    time_stat <- suppressWarnings(sapply(14:ncol(time), function(i) sapply(1:13, function(j) cor.test(time[,i],time[,j],method = "spearman")[c(3,4)])))
    time_cor <- time_stat[seq(2,nrow(time_stat),2),]
    time_pvalue <- time_stat[seq(1,nrow(time_stat)-1,2),]                                                                                                        
    time_max <- sapply(1:(ncol(time)-13), function(i) max(as.numeric(time_cor[,i])))
    time_ident <- sapply(1:(ncol(time)-13), function(i) time_label[which(as.numeric(time_cor[,i])==max(as.numeric(time_cor[,i])))])
    time_maxp <- sapply(1:(ncol(time)-13), function(i) as.numeric(time_pvalue[,i])[which(as.numeric(time_cor[,i])==max(as.numeric(time_cor[,i])))])                     
    names(time_max) <- time_ident

    celltype_stat <- suppressWarnings(sapply(14:ncol(celltype), function(i) sapply(1:13, function(j) cor.test(celltype[,i],celltype[,j],method = "spearman")[c(3,4)])))
    celltype_cor <- celltype_stat[seq(2,nrow(celltype_stat),2),]
    celltype_pvalue <- celltype_stat[seq(1,nrow(celltype_stat)-1,2),]                        
    celltype_max <- sapply(1:(ncol(celltype)-13), function(i) max(as.numeric(celltype_cor[,i])))
    celltype_ident <- sapply(1:(ncol(celltype)-13), function(i) celltype_label[which(as.numeric(celltype_cor[,i])==max(as.numeric(celltype_cor[,i])))])
    celltype_maxp <- sapply(1:(ncol(celltype)-13), function(i) as.numeric(celltype_pvalue[,i])[which(as.numeric(celltype_cor[,i])==max(as.numeric(celltype_cor[,i])))])   
    names(celltype_max) <- celltype_ident

    seucluster.Rad.ID <- celltype_ident
    seucluster.Long.ID <- time_ident
    seucluster.Rad.cor <- celltype_max
    seucluster.Long.cor <- time_max
    seucluster.Rad.pvalue <- celltype_maxp
    seucluster.Long.pvalue <- time_maxp
                             
    time_stat <- suppressWarnings(sapply(14:ncol(time), function(i) sapply(1:13, function(j) cor.test(time[,i],time[,j],method = "pearson")[c(3,4)])))
    time_cor <- time_stat[seq(2,nrow(time_stat),2),]
    time_pvalue <- time_stat[seq(1,nrow(time_stat)-1,2),]                                                                                                        
    time_max <- sapply(1:(ncol(time)-13), function(i) max(as.numeric(time_cor[,i])))
    time_ident <- sapply(1:(ncol(time)-13), function(i) time_label[which(as.numeric(time_cor[,i])==max(as.numeric(time_cor[,i])))])
    time_maxp <- sapply(1:(ncol(time)-13), function(i) as.numeric(time_pvalue[,i])[which(as.numeric(time_cor[,i])==max(as.numeric(time_cor[,i])))])                     
    names(time_max) <- time_ident

    celltype_stat <- suppressWarnings(sapply(14:ncol(celltype), function(i) sapply(1:13, function(j) cor.test(celltype[,i],celltype[,j],method = "pearson")[c(3,4)])))
    celltype_cor <- celltype_stat[seq(2,nrow(celltype_stat),2),]
    celltype_pvalue <- celltype_stat[seq(1,nrow(celltype_stat)-1,2),]                        
    celltype_max <- sapply(1:(ncol(celltype)-13), function(i) max(as.numeric(celltype_cor[,i])))
    celltype_ident <- sapply(1:(ncol(celltype)-13), function(i) celltype_label[which(as.numeric(celltype_cor[,i])==max(as.numeric(celltype_cor[,i])))])
    celltype_maxp <- sapply(1:(ncol(celltype)-13), function(i) as.numeric(celltype_pvalue[,i])[which(as.numeric(celltype_cor[,i])==max(as.numeric(celltype_cor[,i])))])   
    names(celltype_max) <- celltype_ident

    seucluster.Rad.ID.P <- celltype_ident
    seucluster.Long.ID.P <- time_ident
    seucluster.Rad.cor.P <- celltype_max
    seucluster.Long.cor.P <- time_max
    seucluster.Rad.pvalue.P <- celltype_maxp
    seucluster.Long.pvalue.P <- time_maxp
      
    #store values
    rc.ori.ident <- rc.integrated@active.ident

    rc.integrated@active.ident <- rc.ori.ident
    new.cluster.ids <- seucluster.celltype.cor
    names(new.cluster.ids) <- levels(rc.integrated)
    rc.integrated <- RenameIdents(object = rc.integrated, new.cluster.ids)
    rc.integrated@meta.data$seucluster.celltype.cor <- as.numeric(levels(rc.integrated@active.ident)[rc.integrated@active.ident])
    rc.integrated@active.ident <- rc.ori.ident
    new.cluster.ids <- seucluster.celltype.pvalue
    names(new.cluster.ids) <- levels(rc.integrated)
    rc.integrated <- RenameIdents(object = rc.integrated, new.cluster.ids)
    rc.integrated@meta.data$seucluster.celltype.pvalue <- as.numeric(levels(rc.integrated@active.ident)[rc.integrated@active.ident])
    rc.integrated@active.ident <- rc.ori.ident
    new.cluster.ids <- seucluster.celltype.ID
    names(new.cluster.ids) <- levels(rc.integrated)
    rc.integrated <- RenameIdents(object = rc.integrated, new.cluster.ids)
    rc.integrated@meta.data$seucluster.celltype.ID <- as.character(rc.integrated@active.ident)

                             
    rc.integrated@active.ident <- rc.ori.ident
    new.cluster.ids <- seucluster.celltype.cor.P
    names(new.cluster.ids) <- levels(rc.integrated)
    rc.integrated <- RenameIdents(object = rc.integrated, new.cluster.ids)
    rc.integrated@meta.data$seucluster.celltype.cor.P <- as.numeric(levels(rc.integrated@active.ident)[rc.integrated@active.ident])
    rc.integrated@active.ident <- rc.ori.ident
    new.cluster.ids <- seucluster.celltype.pvalue.P
    names(new.cluster.ids) <- levels(rc.integrated)
    rc.integrated <- RenameIdents(object = rc.integrated, new.cluster.ids)
    rc.integrated@meta.data$seucluster.celltype.pvalue.P <- as.numeric(levels(rc.integrated@active.ident)[rc.integrated@active.ident])
    rc.integrated@active.ident <- rc.ori.ident
    new.cluster.ids <- seucluster.celltype.ID.P
    names(new.cluster.ids) <- levels(rc.integrated)
    rc.integrated <- RenameIdents(object = rc.integrated, new.cluster.ids)
    rc.integrated@meta.data$seucluster.celltype.ID.P <- as.character(rc.integrated@active.ident)

    rc.integrated@active.ident <- rc.ori.ident
    new.cluster.ids <- seucluster.timezone.cor
    names(new.cluster.ids) <- levels(rc.integrated)
    rc.integrated <- RenameIdents(object = rc.integrated, new.cluster.ids)
    rc.integrated@meta.data$seucluster.timezone.cor <- as.numeric(levels(rc.integrated@active.ident)[rc.integrated@active.ident])
    rc.integrated@active.ident <- rc.ori.ident
    new.cluster.ids <- seucluster.timezone.pvalue
    names(new.cluster.ids) <- levels(rc.integrated)
    rc.integrated <- RenameIdents(object = rc.integrated, new.cluster.ids)
    rc.integrated@meta.data$seucluster.timezone.pvalue <- as.numeric(levels(rc.integrated@active.ident)[rc.integrated@active.ident])
    rc.integrated@active.ident <- rc.ori.ident
    new.cluster.ids <- seucluster.timezone.ID
    names(new.cluster.ids) <- levels(rc.integrated)
    rc.integrated <- RenameIdents(object = rc.integrated, new.cluster.ids)
    rc.integrated@meta.data$seucluster.timezone.ID <- as.character(rc.integrated@active.ident)
                             
    rc.integrated@active.ident <- rc.ori.ident
    new.cluster.ids <- seucluster.timezone.cor.P
    names(new.cluster.ids) <- levels(rc.integrated)
    rc.integrated <- RenameIdents(object = rc.integrated, new.cluster.ids)
    rc.integrated@meta.data$seucluster.timezone.cor.P <- as.numeric(levels(rc.integrated@active.ident)[rc.integrated@active.ident])
    rc.integrated@active.ident <- rc.ori.ident
    new.cluster.ids <- seucluster.timezone.pvalue.P
    names(new.cluster.ids) <- levels(rc.integrated)
    rc.integrated <- RenameIdents(object = rc.integrated, new.cluster.ids)
    rc.integrated@meta.data$seucluster.timezone.pvalue.P <- as.numeric(levels(rc.integrated@active.ident)[rc.integrated@active.ident])
    rc.integrated@active.ident <- rc.ori.ident
    new.cluster.ids <- seucluster.timezone.ID.P
    names(new.cluster.ids) <- levels(rc.integrated)
    rc.integrated <- RenameIdents(object = rc.integrated, new.cluster.ids)
    rc.integrated@meta.data$seucluster.timezone.ID.P <- as.character(rc.integrated@active.ident)
    
    rc.integrated@active.ident <- rc.ori.ident
    new.cluster.ids <- seucluster.Rad.cor
    names(new.cluster.ids) <- levels(rc.integrated)
    rc.integrated <- RenameIdents(object = rc.integrated, new.cluster.ids)
    rc.integrated@meta.data$seucluster.Rad.cor <- as.numeric(levels(rc.integrated@active.ident)[rc.integrated@active.ident])
    rc.integrated@active.ident <- rc.ori.ident
    new.cluster.ids <- seucluster.Rad.pvalue
    names(new.cluster.ids) <- levels(rc.integrated)
    rc.integrated <- RenameIdents(object = rc.integrated, new.cluster.ids)
    rc.integrated@meta.data$seucluster.Rad.pvalue <- as.numeric(levels(rc.integrated@active.ident)[rc.integrated@active.ident])
    rc.integrated@active.ident <- rc.ori.ident
    new.cluster.ids <- seucluster.Rad.ID
    names(new.cluster.ids) <- levels(rc.integrated)
    rc.integrated <- RenameIdents(object = rc.integrated, new.cluster.ids)
    rc.integrated@meta.data$seucluster.Rad.ID <- as.character(rc.integrated@active.ident)
                             
    rc.integrated@active.ident <- rc.ori.ident
    new.cluster.ids <- seucluster.Rad.cor.P
    names(new.cluster.ids) <- levels(rc.integrated)
    rc.integrated <- RenameIdents(object = rc.integrated, new.cluster.ids)
    rc.integrated@meta.data$seucluster.Rad.cor.P <- as.numeric(levels(rc.integrated@active.ident)[rc.integrated@active.ident])
    rc.integrated@active.ident <- rc.ori.ident
    new.cluster.ids <- seucluster.Rad.pvalue.P
    names(new.cluster.ids) <- levels(rc.integrated)
    rc.integrated <- RenameIdents(object = rc.integrated, new.cluster.ids)
    rc.integrated@meta.data$seucluster.Rad.pvalue.P <- as.numeric(levels(rc.integrated@active.ident)[rc.integrated@active.ident])
    rc.integrated@active.ident <- rc.ori.ident
    new.cluster.ids <- seucluster.Rad.ID.P
    names(new.cluster.ids) <- levels(rc.integrated)
    rc.integrated <- RenameIdents(object = rc.integrated, new.cluster.ids)
    rc.integrated@meta.data$seucluster.Rad.ID.P <- as.character(rc.integrated@active.ident)
                             
    rc.integrated@active.ident <- rc.ori.ident
    new.cluster.ids <- seucluster.Long.cor
    names(new.cluster.ids) <- levels(rc.integrated)
    rc.integrated <- RenameIdents(object = rc.integrated, new.cluster.ids)
    rc.integrated@meta.data$seucluster.Long.cor <- as.numeric(levels(rc.integrated@active.ident)[rc.integrated@active.ident])
    rc.integrated@active.ident <- rc.ori.ident
    new.cluster.ids <- seucluster.Long.pvalue
    names(new.cluster.ids) <- levels(rc.integrated)
    rc.integrated <- RenameIdents(object = rc.integrated, new.cluster.ids)
    rc.integrated@meta.data$seucluster.Long.pvalue <- as.numeric(levels(rc.integrated@active.ident)[rc.integrated@active.ident])
    rc.integrated@active.ident <- rc.ori.ident
    new.cluster.ids <- seucluster.Long.ID
    names(new.cluster.ids) <- levels(rc.integrated)
    rc.integrated <- RenameIdents(object = rc.integrated, new.cluster.ids)
    rc.integrated@meta.data$seucluster.Long.ID <- as.character(rc.integrated@active.ident)
                             
    rc.integrated@active.ident <- rc.ori.ident
    new.cluster.ids <- seucluster.Long.cor.P
    names(new.cluster.ids) <- levels(rc.integrated)
    rc.integrated <- RenameIdents(object = rc.integrated, new.cluster.ids)
    rc.integrated@meta.data$seucluster.Long.cor.P <- as.numeric(levels(rc.integrated@active.ident)[rc.integrated@active.ident])
    rc.integrated@active.ident <- rc.ori.ident
    new.cluster.ids <- seucluster.Long.pvalue.P
    names(new.cluster.ids) <- levels(rc.integrated)
    rc.integrated <- RenameIdents(object = rc.integrated, new.cluster.ids)
    rc.integrated@meta.data$seucluster.Long.pvalue.P <- as.numeric(levels(rc.integrated@active.ident)[rc.integrated@active.ident])
    rc.integrated@active.ident <- rc.ori.ident
    new.cluster.ids <- seucluster.Long.ID.P
    names(new.cluster.ids) <- levels(rc.integrated)
    rc.integrated <- RenameIdents(object = rc.integrated, new.cluster.ids)
    rc.integrated@meta.data$seucluster.Long.ID.P <- as.character(rc.integrated@active.ident)
    

In [16]:
DF <- c()
for (i in grep("DF.classifications",colnames(rc.integrated@meta.data))){
    DF <- c(DF, rc.integrated@meta.data[,i][!is.na(rc.integrated@meta.data[,i])])
}
rc.integrated@meta.data$DF.classifications <- DF

In [21]:
saveRDS(rc.integrated, file = "./Integration/BR_time_course.rds")